# Managers and QuerySets

> In Django, **managers** and **querysets** play a crucial role in retrieving data from the database, managing queries, and encapsulating database logic. Understanding how to effectively use and customize them can greatly improve the maintainability and performance of your Django applications.


- skip_showdoc: true
- skip_exec: true

### 1. **What are Managers in Django?**

A **manager** in Django is the interface through which **database query operations** are provided to Django models. Every model in Django has at least one manager, and by default, Django provides a `Manager` object called `objects`.

#### a. **Default Manager**
By default, every model has a manager named `objects`, which allows you to interact with the model’s database records.

```python
# Example model
class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=100)
    published_date = models.DateField()

# Using the default manager to query the database
books = Book.objects.all()  # Retrieves all books
book = Book.objects.get(id=1)  # Retrieves a specific book by ID
```

#### b. **Custom Manager**
You can create a custom manager to encapsulate logic specific to your model, allowing for more readable and reusable code.

```python
# Custom manager
class BookManager(models.Manager):
    def published_last_year(self):
        last_year = timezone.now().year - 1
        return self.filter(published_date__year=last_year)

# Using a custom manager
class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=100)
    published_date = models.DateField()

    objects = BookManager()  # Use the custom manager

# Using the custom manager method
last_year_books = Book.objects.published_last_year()
```

### 2. **The Role of QuerySets in Django**

A **QuerySet** represents a collection of database queries that return a set of results. QuerySets are lazily evaluated, meaning that the database query is only executed when the QuerySet is evaluated (e.g., when iterating over it or converting it to a list).

#### a. **Basic QuerySet Operations**
QuerySets allow you to filter, order, and retrieve data from the database.

```python
# Retrieving all objects
books = Book.objects.all()

# Filtering objects
filtered_books = Book.objects.filter(author="George Orwell")

# Chaining filters (AND logic)
recent_books = Book.objects.filter(published_date__year=2020).filter(author="George Orwell")

# Retrieving a single object
book = Book.objects.get(id=1)

# Exclude objects
non_orwell_books = Book.objects.exclude(author="George Orwell")

# Ordering results
ordered_books = Book.objects.order_by('published_date')

# Limit number of results (slicing)
first_ten_books = Book.objects.all()[:10]
```

#### b. **QuerySet Evaluation**
A QuerySet is **lazily evaluated**, meaning the query is not actually executed in the database until the QuerySet is iterated over or explicitly evaluated.

Examples of when QuerySets are evaluated:
- **Iterating** over the QuerySet.
- **Slicing** the QuerySet.
- **Serializing** the QuerySet (e.g., converting it to a list or calling `len()`).

```python
# Query is not executed yet
books = Book.objects.filter(author="George Orwell")

# Query is executed when you evaluate the QuerySet
for book in books:
    print(book.title)
```

### 3. **Creating Custom Managers**

Django allows you to define custom managers to encapsulate common queries, making your code more modular and readable. A custom manager can be defined by subclassing `models.Manager`.

#### a. **Basic Custom Manager**

```python
class PublishedBookManager(models.Manager):
    def get_queryset(self):
        # Override the default get_queryset method to return only published books
        return super().get_queryset().filter(published_date__isnull=False)

# Using the custom manager in a model
class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=100)
    published_date = models.DateField()

    # Define two managers: a default manager and a custom manager
    objects = models.Manager()  # Default manager
    published = PublishedBookManager()  # Custom manager for published books
```

In this example:
- `Book.objects.all()` will return all books.
- `Book.published.all()` will return only books that have a `published_date`.

#### b. **Multiple Managers**
You can define multiple managers in a single model to access different sets of query logic.

```python
class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=100)
    published_date = models.DateField()

    # Default and custom managers
    objects = models.Manager()  # Default manager
    published = PublishedBookManager()  # Custom manager
```

### 4. **Custom QuerySets**

Instead of overriding the manager, you can create a custom QuerySet and use it directly in your model manager. This method allows you to chain custom QuerySet methods.

#### a. **Creating a Custom QuerySet**

```python
class BookQuerySet(models.QuerySet):
    def published(self):
        return self.filter(published_date__isnull=False)

    def by_author(self, author_name):
        return self.filter(author=author_name)

# Use the custom QuerySet in a manager
class BookManager(models.Manager):
    def get_queryset(self):
        return BookQuerySet(self.model, using=self._db)

# Use the manager in the model
class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=100)
    published_date = models.DateField()

    objects = BookManager()
```

Now you can chain custom QuerySet methods together:

```python
books = Book.objects.published().by_author("George Orwell")
```

#### b. **Chaining Custom QuerySet Methods**
Custom QuerySet methods allow you to chain operations and write more readable and reusable query logic:

```python
books = Book.objects.published().by_author("J.K. Rowling").order_by('published_date')
```

#### c. **Combining Managers and QuerySets**

A common pattern is to define both custom QuerySets and managers, allowing you to use manager-level logic while preserving the ability to chain QuerySet methods.

```python
class BookQuerySet(models.QuerySet):
    def published(self):
        return self.filter(published_date__isnull=False)

class BookManager(models.Manager):
    def get_queryset(self):
        return BookQuerySet(self.model, using=self._db)

    def published_last_year(self):
        last_year = timezone.now().year - 1
        return self.get_queryset().published().filter(published_date__year=last_year)
```

### 5. **Manager Methods vs QuerySet Methods**

- **Manager Methods**: Manager methods are useful when you want to return data that does not require chaining or modifying the base QuerySet.
- **QuerySet Methods**: QuerySet methods are ideal when you want to create reusable filtering logic that can be chained with other QuerySet methods.

#### When to Use Each:
- Use **manager methods** when the method represents an action that doesn’t require chaining (e.g., creating objects or counting objects).
- Use **custom QuerySet methods** when you want to filter or manipulate a set of results and keep the option of chaining other QuerySet methods.

### 6. **QuerySet Methods**

Django’s QuerySet API provides numerous built-in methods to filter, manipulate, and aggregate data. Some of the most commonly used methods are:

#### a. **`all()`**
Returns all records of the model.
```python
Book.objects.all()
```

#### b. **`filter()`**
Filters the records based on given conditions.
```python
Book.objects.filter(author="George Orwell")
```

#### c. **`exclude()`**
Excludes records that match the given conditions.
```python
Book.objects.exclude(author="George Orwell")
```

#### d. **`get()`**
Retrieves a single object. Raises `DoesNotExist` if no object is found and `MultipleObjectsReturned` if more than one object is found.
```python
Book.objects.get(id=1)
```

#### e. **`order_by()`**
Orders the results by the given field(s). You can specify ascending or descending order by adding a `-` before the field name.
```python
Book.objects.order_by('published_date')
Book.objects.order_by('-published_date')  # Descending order
```

#### f. **`values()`**
Returns a QuerySet of dictionaries, each representing an object with field-value pairs.
```python
Book.objects.values('title', 'author')
```

#### g. **`values_list()`**
Similar to `values()`, but returns a list of tuples.
```python
Book.objects.values_list('title', 'author')
```

#### h. **`distinct()`**
Eliminates duplicate rows from the query results.
```python
Book.objects.distinct('author')
```

#### i. **`count()`**
Returns the number of records that match the query.
```python
Book.objects.filter(author="George Orwell").count()
```

#### j. **`aggregate()`**
Performs aggregation (e.g., `Sum`, `Count`, `Avg`, `Min`, `Max`).
```python
from django.db.models import Avg

Book.objects.aggregate(Avg('published_date'))
```

#### k. **`exists()`**
Returns `True` if the QuerySet contains any results

, `False` otherwise.
```python
Book.objects.filter(author="George Orwell").exists()
```

#### l. **`annotate()`**
Adds additional fields to each result in the QuerySet. Often used for aggregation.
```python
from django.db.models import Count

Book.objects.annotate(total_books=Count('id'))
```

#### m. **`select_related()`**
Optimizes queries by performing an SQL `JOIN` and including related objects in the result.
```python
Book.objects.select_related('author')
```

#### n. **`prefetch_related()`**
Optimizes queries for `ManyToMany` and `Reverse ForeignKey` relationships by performing multiple separate queries instead of a `JOIN`.
```python
Book.objects.prefetch_related('categories')
```

### 7. **Manager and QuerySet Best Practices**

#### a. **Keep Business Logic in Managers and QuerySets**
Encapsulate complex queries or business logic inside custom managers and QuerySets to make your views and templates cleaner.

#### b. **Use `get_queryset()` Wisely**
Always return a `QuerySet` from the `get_queryset()` method, as it allows you to further modify the query (e.g., chaining filters, ordering).

#### c. **Leverage Lazy Evaluation**
Django QuerySets are lazily evaluated, so you can build complex queries without immediately hitting the database. However, ensure queries are evaluated efficiently by minimizing unnecessary evaluations.

#### d. **Avoid Logic in Views**
Instead of placing filtering and querying logic in views, use custom managers or QuerySets to encapsulate and reuse this logic across your application.

#### e. **Chaining Queries**
When creating custom QuerySet methods, ensure they return a QuerySet so that you can continue to chain additional filters and operations.